In [58]:
import io
import contextlib
from dotenv import load_dotenv

load_dotenv()

True

In [59]:
import os
os.environ.get("API_KEY") or "no-set"
OPENAI_KEY = os.environ.get("OPEN_API_KEY")
assert OPENAI_KEY, "Missing OPEN_API_KEY in .env"

In [60]:
BRIGHT_DATA_SERP_API_KEY = os.environ.get("BRIGHT_DATA_SERP_API_KEY") 
assert BRIGHT_DATA_SERP_API_KEY is not None

In [61]:
from langchain_brightdata import BrightDataSERP
serp_tool = BrightDataSERP(
    bright_data_api_key=BRIGHT_DATA_SERP_API_KEY,
    zone = "serp_api3",
    search_engine="google",
    country="us",
    parse_results =True
)

In [62]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-5-nano",
    api_key=OPENAI_KEY
)

In [63]:
tools = [serp_tool]


In [64]:
tool_mapping ={
    x.name: x for x in tools
}

In [65]:
llm_with_tools = model.bind_tools(tools)

In [66]:
llm_with_tools.invoke("when was python made?")


AIMessage(content='- Python was created by Guido van Rossum, starting in December 1989.\n- The first public release was Python 0.9.0, released on February 20, 1991.\n- Python 1.0 followed in January 1994.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 447, 'prompt_tokens': 547, 'total_tokens': 994, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DEDi97B8HjqSojxATV4uMeXhuIAQ9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ca438-29a2-77d3-b309-48ae3bd15c7a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 547, 'output_tokens': 447, 'total_tokens': 994, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'outp

In [68]:
# ai_msg = llm_with_tools.invoke("when was python made?")
ai_msg = llm_with_tools.invoke(
    "Search Google for the current CEO of OpenAI and return the result."
)

ai_msg.tool_calls
for tool_call_val  in ai_msg.tool_calls:
    func_name = tool_call_val.get("name")
    func_args = tool_call_val.get("args")
    _tool = tool_mapping.get(func_name)
    _id = tool_call_val.get("id")
    print(func_name, func_args, _tool)
    

brightdata_serp {'query': 'current CEO of OpenAI', 'zone': 'serp', 'search_engine': 'google', 'country': 'us', 'language': 'en', 'results_count': 5, 'search_type': None, 'device_type': None, 'parse_results': True} zone='serp_api3' parse_results=True api_wrapper=BrightDataSERPAPIWrapper(bright_data_api_key=SecretStr('**********'))


In [72]:
#tool calling in langgraph
from langchain_core.messages import ToolMessage

def ask_agent(question: str):
    ai_msg = llm_with_tools.invoke(question)
    messages = [ai_msg]

    if ai_msg.tool_calls:
        for tool_call in ai_msg.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            tool = tool_mapping.get(tool_name)

            tool_result = tool.invoke(tool_args)

            messages.append(
                ToolMessage(
                    content=str(tool_result),
                    tool_call_id=tool_call["id"]
                )
            )

        final_response = llm_with_tools.invoke(messages)
        return final_response.content

    return ai_msg.content

In [73]:
ask_agent("Search Google for latest Tesla stock price.")

''

In [92]:
import langchain
import langchain_core
import langchain_openai

print("langchain:", getattr(langchain, "__version__", "unknown"))
print("langchain_core:", getattr(langchain_core, "__version__", "unknown"))
print("langchain_openai:", getattr(langchain_openai, "__version__", "unknown"))

langchain: 1.2.10
langchain_core: 1.2.15
langchain_openai: unknown


In [99]:
import json, time
from langchain_core.tools import Tool
from langchain_core.messages import ToolMessage

# Wrapper tool to prevent zone overrides + keep your serp_api3 config
def google_search(query: str) -> str:
    return serp_tool.invoke(query)

search_tool = Tool(
    name="google_search",
    description="Search Google via BrightData SERP. Input is a plain query string.",
    func=google_search,
)

tools = [search_tool]
tool_mapping = {t.name: t for t in tools}

# Call #1 can tool-call
llm_with_tools = model.bind_tools(tools, tool_choice="any")

def ask_agent_once(question: str) -> str:
    # LLM call #1
    ai_msg = llm_with_tools.invoke(
        f"Use google_search exactly once. Search query should be short. "
        f"Then wait for results.\nQuestion: {question}"
    )

    if not getattr(ai_msg, "tool_calls", None):
        return ai_msg.content or "(no content returned in first call)"

    messages = [ai_msg]

    # Execute tool
    for call in ai_msg.tool_calls:
        tool_name = call["name"]
        tool_args = call["args"]
        tool_id = call["id"]

        if isinstance(tool_args, dict):
            query = tool_args.get("query") or tool_args.get("input") or next(iter(tool_args.values()))
        else:
            query = str(tool_args)

        tool_result = tool_mapping[tool_name].invoke(query)

        messages.append(
            ToolMessage(
                content=tool_result if isinstance(tool_result, str)
                        else json.dumps(tool_result, ensure_ascii=False),
                tool_call_id=tool_id
            )
        )

    time.sleep(0.7)  # small pause to avoid burst rate limits

    # LLM call #2 (NO tools)
    final = model.invoke(
        messages + [
            {"role": "user", "content": "Answer using ONLY the search results above. "
                                        "Return a short answer and include one source link. "
                                        "Do NOT call any tools."}
        ]
    )

    return final.content or "(no final content returned)"

In [100]:
ask_agent_once("Find the latest Tesla (TSLA) stock price and cite the source.")


'Tesla stock (TSLA) is around 402.51 USD per share today. Source: https://finance.yahoo.com/quote/TSLA/'